## Extracting_User_Data Using Langchain Document Loader (res:- fail)

In [ ]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
pip install Selenium unstructured -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 11.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import SeleniumURLLoader

In [ ]:
loader = SeleniumURLLoader(urls=['https://www.reddit.com/r/BollyBlindsNGossip/'], continue_on_failure=True)

In [ ]:
loader.load()

[Document(metadata={'source': 'https://www.reddit.com/r/BollyBlindsNGossip/', 'title': 'No title found.', 'description': 'No description found.', 'language': 'No language found.'}, page_content="\n\nYou've been blocked by network security.\n\nTo continue, log in to your Reddit account or use your developer token If you think you've been blocked by mistake, file a ticket below and we'll look into it.\n\nLog in File a ticket")]

## Tried Using reddit api (res :- Pass)

In [ ]:
pip install praw

In [ ]:
import praw

reddit = praw.Reddit(
    client_id="Put Your ID",
    client_secret='Put Your Secret',
    user_agent = 'Agent Name',
    check_for_async=False
)

In [ ]:
print(reddit.read_only)

True


In [ ]:
username = "United_Pineapple_932 "  # Replace with the Reddit username you want to analyze

redditor = reddit.redditor(username)

In [ ]:
posts_dict = {}
comments_dict = {}

# Fetch up to 50 posts
for idx, submission in enumerate(redditor.submissions.new(limit=50), 1):
    posts_dict[f"post_{idx}"] = {
        "title": submission.title,
        "selftext": submission.selftext,
        "subreddit": str(submission.subreddit),
        "created_utc": submission.created_utc,
        "link": f"https://www.reddit.com{submission.permalink}"
    }

In [ ]:
for idx, comment in enumerate(redditor.comments.new(limit=50), 1):
    comments_dict[f"comment_{idx}"] = {
        "body": comment.body,
        "subreddit": str(comment.subreddit),
        "created_utc": comment.created_utc,
        "link": f"https://www.reddit.com{comment.permalink}"
    }


In [ ]:
posts_dict_llm = {
    k: {key: val for key, val in v.items() if key != "link"}
    for k, v in posts_dict.items()
}
comments_dict_llm = {
    k: {key: val for key, val in v.items() if key != "link"}
    for k, v in comments_dict.items()
}

## Creating a Gemini Client

In [ ]:
pip install google-ai-generativelanguage==0.6.15

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.7 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
user_data_for_llm = {}

for key, value in user_data.items():
    value_no_link = {k: v for k, v in value.items() if k != "link"}
    user_data_for_llm[key] = value_no_link

In [ ]:
from google import genai

In [ ]:
post_prompt = (
    "Analyze the following Reddit posts. For each, note any interests, personality traits, or communication styles you observe. "
    "Summarize the user's characteristics based only on these posts. "
    "For each trait, cite the title or a snippet from the relevant post.\n\n"
    f"{posts_dict_llm}"
)


In [ ]:
comment_prompt = (
    "Analyze the following Reddit comments. For each, note any interests, personality traits, or communication styles you observe. "
    "Summarize the user's characteristics based only on these comments. "
    "For each trait, cite a snippet from the relevant comment.\n\n"
    f"{comments_dict_llm}"
)


In [ ]:
# user_content = ""
# for key, value in user_data_for_llm.items():
#     if value["type"] == "comment":
#         user_content += f"[Comment] (Subreddit: {value['subreddit']}, Time: {value['created_utc']}): {value['body']}\n\n"
#     elif value["type"] == "post":
#         user_content += f"[Post] (Subreddit: {value['subreddit']}, Time: {value['created_utc']}): Title: {value['title']}\n{value['selftext']}\n\n"

# # Create a short, clear prompt
# prompt = (
#     f"Based on the following Reddit posts and comments, generate a brief persona for the user. "
#     f"Summarize their interests, communication style, and any notable traits.\n\n"
#     f"{user_content}"
# )

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('gemini2')

In [ ]:
client = genai.Client(api_key = API_KEY)

In [ ]:
response_prompt = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=post_prompt,
)

In [ ]:
response_comment = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=comment_prompt,
)

In [ ]:
synthesis_prompt = (
    "Based on the analyses of the user's posts and comments below, synthesize a comprehensive persona. "
    "Integrate traits from both sources, and ensure each trait is supported by a cited post or comment.\n\n"
    f"Post analysis:\n{response_prompt.text}\n\n"
    f"Comment analysis:\n{response_comment.text}\n"
)

In [ ]:
response_persona = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=synthesis_prompt,
)

In [ ]:
response_persona.text

'Based on the comprehensive analysis of the user\'s posts and comments, a detailed persona emerges:\n\n---\n\n## Persona: The Inquisitive Indian Chronicler\n\n**Core Identity:**\n"The Inquisitive Indian Chronicler" is a highly engaged, intellectually curious, and meticulously detail-oriented Reddit user with a profound passion for Indian history, culture, and national achievements. They are a dedicated digital archivist and educator, often sharing vast amounts of information and context, while also demonstrating a strong sense of social justice and a sharp, sometimes sarcastic, wit.\n\n---\n\n### Key Interests:\n\n*   **Profound Interest in Indian History & Culture:**\n    *   **Ancient Civilizations & Archaeology:** Deeply interested in ancient Indian civilizations (Harappan, Indus Valley, Mohenjo-daro), their artifacts, and archaeological theories, including potential links with other ancient cultures. (e.g., Post 4: "Question: How did such delicate artefacts survive thousands of yea